In [ ]:
#all python packages
import pandas as pd
from pprint import pprint
import os
import feature_extraction_soccer_analysis

import datasets

In [ ]:
DATA_PATH = r'../data'

In [ ]:
dfplayer = pd.read_json(os.path.join(DATA_PATH, 'players_new.json'),encoding = "unicode-escape")
dfteam = pd.read_json(os.path.join(DATA_PATH, 'teams_new.json'),encoding = "unicode-escape")
dfmatc = pd.read_json(os.path.join(DATA_PATH, 'matches_Italy.json'), encoding = "unicode-escape")
dfevent = pd.read_json(os.path.join(DATA_PATH, 'events_Italy.json'), encoding = "unicode-escape")

In [ ]:
#we want only italian serie a competitions id 524
SERIE_A = 524
dfmatc = dfmatc[dfmatc['competitionId'] == SERIE_A]
dfmatc = dfmatc.reindex(sorted(dfmatc.columns), axis=1)
dfevent = dfevent.reindex(sorted(dfevent.columns), axis=1)
dfplayer = dfplayer.reindex(sorted(dfplayer.columns), axis = 1)
dfteam = dfteam.reindex(sorted(dfteam.columns), axis=1)

extract players that played during matches of the season. 
**Note**: order of the columns is important. 

In [ ]:
dfToFill = feature_extraction_soccer_analysis.playerMatchExtraction(dfmatc)
dfToFill = dfToFill[['match_id', 'gameweek', 'team_id', 'playerId', 'minutes_played', 'red_card', 'yellow_card', 'goals']]

In [ ]:
dfToFill.head()

extract quantity and quality features (plus dangerous ones). Before merging the dangerous **is needed to be computed the contribution one**

In [ ]:
dfToFill, dfDangerousFeatures = feature_extraction_soccer_analysis.qualityFeaturesComputationManager(dfToFill, dfevent)

#### Replace Null in Goals

In [ ]:
newGoal = []
for player in dfToFill.values:
    if(player[7] == 'null'):
        newGoal.append(0)
    else:
        newGoal.append(player[7])
dfToFill['goals'] = newGoal

#### Quality Features

In [ ]:
totalQuality = feature_extraction_soccer_analysis.computeTotalForEachMatchTeam(dfToFill)

In [ ]:
dfToFill = feature_extraction_soccer_analysis.computeContributionFeatures(dfToFill, totalQuality)

In [ ]:
dfToFill

#### Add the deangerous features already computed

In [ ]:
dfDangerousFeatures.rename(columns={'id':'playerId'}, inplace=True)
dfDangerousFeatures.rename(columns={'match':'match_id'}, inplace=True)
dfToFill = pd.merge(dfToFill, dfDangerousFeatures, on=['match_id', 'playerId'])
dfToFill.rename(columns={'playerId':'player_id'}, inplace=True)

#### Contextual Features Computation

In [ ]:
dfToCompletion = dfToFill.copy(deep=True)

In [ ]:
dfToCompletion = feature_extraction_soccer_analysis.meanPassesRecoil(dfToCompletion, dfevent)

In [ ]:
dfToCompletion = feature_extraction_soccer_analysis.centerOfGravity(dfToCompletion, dfevent)

In [ ]:
dfToCompletion = feature_extraction_soccer_analysis.computeEjectionFeature(dfToCompletion)

In [ ]:
dfToCompletion = feature_extraction_soccer_analysis.computeCountryOfThePlayer(dfToCompletion, dfplayer)

In [ ]:
dfToCompletion = feature_extraction_soccer_analysis.computeBigMatchFeature(dfToCompletion, dfmatc)

In [ ]:
dfToCompletion = feature_extraction_soccer_analysis.computeLastContextualVariables(dfToCompletion, dfmatc, dfteam)

In [ ]:
dfToCompletion = feature_extraction_soccer_analysis.computeAgeForPlayer(dfToCompletion, dfplayer)

In [ ]:
dfBetting = pd.read_csv(os.path.join(DATA_PATH, 'betting.csv'))

In [ ]:
dfToCompletion = feature_extraction_soccer_analysis.computeExpectation(dfToCompletion, dfBetting)

In [ ]:
dfToCompletion.head(1)

change the path based on the season

In [ ]:
dfToCompletion.to_csv(os.path.join(DATA_PATH, 'dataset_player_match_season_2017_2018_featuresextracted.csv'), index = False)